<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
from janitor import clean_names

In [3]:
# Reading the json as a dict
with open('data/scraped_fines.json') as json_data:
    fines = json.load(json_data)

In [4]:
fines_df = (
    pd
    .json_normalize(fines['allItems'])
    .rename(columns ={'articleViolated':'article_violated'})
)

In [5]:
fines_df

,id,picture,name,price,authority,date,controller,article_violated,type,source,summary
0,1,https://www.privacyaffairs.com/wp-content/uplo...,Poland,9380,Polish National Personal Data Protection Offic...,10/18/2019,Polish Mayor,Art. 28 GDPR,Non-compliance with lawful basis for data proc...,https://uodo.gov.pl/decyzje/ZSPU.421.3.2019,No data processing agreement has been conclude...
1,2,https://www.privacyaffairs.com/wp-content/uplo...,Romania,2500,Romanian National Supervisory Authority for Pe...,10/17/2019,UTTIS INDUSTRIES,"Art. 12 GDPR, Art. 13 GDPR, Art. 5 (1) c) GDPR...",Information obligation non-compliance,https://www.dataprotection.ro/?page=A_patra_am...,A controller was sanctioned because he had unl...
2,3,https://www.privacyaffairs.com/wp-content/uplo...,Spain,60000,Spanish Data Protection Authority (AEPD),10/16/2019,Xfera Moviles S.A.,"Art. 5 GDPR, Art. 6 GDPR",Non-compliance with lawful basis for data proc...,https://www.aepd.es/resoluciones/PS-00262-2019...,The company had unlawfully processed the perso...
3,4,https://www.privacyaffairs.com/wp-content/uplo...,Spain,8000,Spanish Data Protection Authority (AEPD),10/16/2019,Iberdrola Clientes,Art. 31 GDPR,Failure to cooperate with supervisory authority,https://www.aepd.es/resoluciones/PS-00304-2019...,Iberdrola Clientes violated Article 13 of the ...
4,5,https://www.privacyaffairs.com/wp-content/uplo...,Romania,150000,Romanian National Supervisory Authority for Pe...,10/09/2019,Raiffeisen Bank SA,Art. 32 GDPR,Failure to implement sufficient measures to en...,https://www.dataprotection.ro/?page=Comunicat_...,Raiffeisen Bank Romania did not observe the ne...
...,...,...,...,...,...,...,...,...,...,...,...
875,876,https://www.privacyaffairs.com/wp-content/uplo...,Spain,56000,Spanish Data Protection Authority (AEPD),09/14/2021,"Vodafone Espana, S.A.U.",Art. 6 (1) GDPR,Non-compliance with lawful basis for data proc...,,The Spanish DPNA (AEPD) has fined Vodafone Esp...
876,877,https://www.privacyaffairs.com/wp-content/uplo...,Spain,56000,Spanish Data Protection Authority (AEPD),09/14/2021,"Vodafone Espana, S.A.U.",Art. 6 (1) GDPR,Non-compliance with lawful basis for data proc...,,The Spanish DPA (also known as AEPD) fined Vo...
877,878,https://www.privacyaffairs.com/wp-content/uplo...,Spain,4000,Spanish Data Protection Authority (AEPD),09/16/2021,Frigorifica Botana S.L.,Art. 5 (1) c) GDPR,Failure to comply with data processing principles,,The Spanish DPA (also known as AEPD) has fined...
878,879,https://www.privacyaffairs.com/wp-content/uplo...,Spain,18000,Spanish Data Protection Authority (AEPD),09/20/2021,CEDICO,Art. 5 (1) f) GDPR,Failure to comply with data processing principles,,"The Spanish DPA (AEPD) has fined CEDICO, CENTR..."


In [6]:
art_text = (
    pd
    .read_json('data/scraped_art_text.json')
    .T
    .reset_index()
    .rename({'index':'chapter', 'title':'chapter_title', 'link':'chapter_link'}
            ,axis='columns')
    .assign(chapter=lambda x: x.chapter.str.extract(r'(\d+)').astype('int'))
)

In [207]:
# empty df, loop will add rows to the bottom
long = pd.DataFrame()

for i in range(len(art_text.chapter)):
    unnested_articles = (
        pd
        .DataFrame(art_text.articles[i])
        .T
        .assign(chapter=i + 1)
    )
    
    sub_articles = (
        unnested_articles
        .assign(text=lambda x: x.text.str.replace(r'((?=\n[\d*A-Z]).+?)', '\t', regex=True))
        .text.str.split(r'\t', expand=True)
        .apply(squeeze, axis=1)
        .T
        .melt(id_vars=None, value_name='text', var_name='article')
        .dropna()
        .assign(sub_art=lambda x: x.groupby('article').cumcount()+1)
        .reset_index(drop=True)
    )
        
    long_unnested = (
        unnested_articles
        .drop(columns='text')
        .reset_index()
        .merge(sub_articles, left_on='index', right_on='article')
        .loc[:,['chapter', 'title', 'link', 'article','sub_art', 'text']]
    )
    
    # finally add this to initialised df
    long=pd.concat([long, long_unnested], axis='rows')



In [208]:
long

,chapter,title,link,article,sub_art,text
0,1,Subject-matter and objectives,http://gdpr-info.eu/art-1-gdpr/,1,1,\nThis Regulation lays down rules relating to ...
1,1,Material scope,http://gdpr-info.eu/art-2-gdpr/,2,1,\nThis Regulation applies to the processing of...
2,1,Territorial scope,http://gdpr-info.eu/art-3-gdpr/,3,1,\nThis Regulation applies to the processing of...
3,1,Definitions,http://gdpr-info.eu/art-4-gdpr/,4,1,\n‘personal data’ means any information relati...
0,2,Principles relating to processing of personal ...,http://gdpr-info.eu/art-5-gdpr/,5,1,\nPersonal data shall be:\n\nprocessed lawfull...
...,...,...,...,...,...,...
1,11,Relationship with Directive 2002/58/EC,http://gdpr-info.eu/art-95-gdpr/,95,1,This Regulation shall not impose additional ob...
2,11,Relationship with previously concluded Agreements,http://gdpr-info.eu/art-96-gdpr/,96,1,International agreements involving the transfe...
3,11,Commission reports,http://gdpr-info.eu/art-97-gdpr/,97,1,\n1By 25 May 2020 and every four years thereaf...
4,11,Review of other Union legal acts on data prote...,http://gdpr-info.eu/art-98-gdpr/,98,1,"1The Commission shall, if appropriate, submit ..."


In [215]:
(
    pd
    .DataFrame(art_text.articles[0])
    .T
    .assign(chapter=1 + 1)
    .text
).replace(r'(?=\n\d*[^a-z1-9]).+?', r'\t')

1    \nThis Regulation lays down rules relating to ...
2    \nThis Regulation applies to the processing of...
3    \nThis Regulation applies to the processing of...
4    \n‘personal data’ means any information relati...
Name: text, dtype: object

sub sub articles !!! arg. need to look for lower case laters at the beginning of text. if so, add another bit a, b, ... required

In [180]:
import string

In [183]:
letters = [letter for letter in string.ascii_lowercase]

In [186]:
sub_articles

,article,text,sub_art
0,94,Directive 95/46/EC is repealed with effect fro...,1
1,94,1References to the repealed Directive shall be...,2
2,95,This Regulation shall not impose additional ob...,1
3,96,International agreements involving the transfe...,1
4,97,1By 25 May 2020 and every four years thereafte...,1
5,97,In the context of the evaluations and reviews ...,2
6,97,Chapter V on the transfer of personal data to ...,3
7,97,Chapter VII on cooperation and consistency.,4
8,97,"For the purpose of paragraph 1, the Commission...",5
9,97,In carrying out the evaluations and reviews re...,6


In [63]:
def squeeze(x):
    original_columns = x.index.tolist()

    pre = x.replace({'':np.nan})
    squeezed = pre.dropna()
    squeezed.index = [original_columns[n] for n in range(squeezed.count())]

    return squeezed.reindex(original_columns, fill_value=np.nan).dropna()